# This file contains code to find recipes that are similar to other recipes in our database

## Load the recipes database

In [135]:
# Import pandas
import pandas as pd

In [136]:
RD = pd.read_pickle('./Recipes_DataFrame.pkl')

# Look at the database
RD.head()



,Classification,Ingredients and Instructions,Name
0,0,water butter salt pepper all purpose flour...,Bacon Cheese Puff Balls
1,0,"dates, pitted bacon, lean, cut in half width-...",Bacon Wrapped Sherried Dates
2,0,zucchini kosher salt French lentils very fr...,Baked Stuffed Portabellini Mushrooms
3,0,"sour cream, lower fat mayonnaise garlic, min...",Buffalo Brussels Sprouts
4,0,cans chicken broth boneless pork shoulder ch...,Carnitas


In [137]:
# convert ingredients and instructions to lower case
RD['Ingredients and Instructions'] = RD['Ingredients and Instructions'].\
                                        apply(lambda row: row.lower())
RD.head()


import string
# remove punctuations using the character deletion step of translate
RD['Ingredients and Instructions'] = RD['Ingredients and Instructions'].\
apply(lambda row: row.translate(None, string.punctuation))
RD.head()

,Classification,Ingredients and Instructions,Name
0,0,water butter salt pepper all purpose flour...,Bacon Cheese Puff Balls
1,0,dates pitted bacon lean cut in half widthwise...,Bacon Wrapped Sherried Dates
2,0,zucchini kosher salt french lentils very fr...,Baked Stuffed Portabellini Mushrooms
3,0,sour cream lower fat mayonnaise garlic mince...,Buffalo Brussels Sprouts
4,0,cans chicken broth boneless pork shoulder ch...,Carnitas


## Calculate tf-idf (term frequency - inverse document frequency)
Tf-idf reflects how important a word (ingredient or instruction) is in a recipe compared to all other recipes

In [138]:
# import some modules for calculating tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize

In [139]:
# write a function that will tokenize and stem words
stemmer = PorterStemmer()
def tokenize(text):
    tokens = word_tokenize(text)
    stems = []
    for token in tokens:
        stems.append(stemmer.stem(token))
    return stems

In [140]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')

# Obtain a matrix with tf-idfs for different recipes. 
# The columns of the matrix will contain features, 
# which are the weighted frequency of tokens
tfidf_mat = tfidf.fit_transform(RD['Ingredients and Instructions'])

## Remove rows that are entirely zero

In [146]:
import numpy as np
# remove rows that are entirely zero from the tfidf matrix and also from recipes names
num_nonzeros =  np.diff(tfidf_mat.indptr)
zeros_ind = np.nonzero(num_nonzeros == 0)[0][0]
tfidf_mat =  tfidf_mat[num_nonzeros != 0]

print "zeros_ind = ", zeros_ind

#Also remove those recipes from the recipes databse
RD = RD.drop(RD.index[zeros_ind])

IndexError: index 0 is out of bounds for axis 0 with size 0

## Calculate the cosine similarity between recipes

In [153]:
# import module to calculate distances between vectors
from sklearn.metrics.pairwise import cosine_similarity

In [154]:
# similarity[i,j] will store how far recipe i is from recipe j
similarity = cosine_similarity(tfidf_mat)

## Look at a couple of examples of similar recipes

In [144]:
key_lime_bars = RD[RD['Name'] == 'Key Lime Bars'].index
key_lime_cookies = RD[RD['Name'] == 'Key Lime Cookies'].index
brussels_sprouts = RD[RD['Name'] == 'Buffalo Brussels Sprouts'].index

# for sanity check, make sure that a recipe is 100% similar to itself
print "Similarity between key lime bars and itself = ", similarity[key_lime_bars, key_lime_bars]
print "Similarity between key lime cookies and itself = ", similarity[key_lime_cookies, key_lime_cookies]
print "Similarity between buffalo brussels sprouts and itself = ", similarity[brussels_sprouts, brussels_sprouts]

# also check how similar these are to one another
print "Similarity between key lime bars and key lime cookies = ", \
similarity[key_lime_bars, key_lime_cookies]
print "Similarity between key lime bars and buffalo brussels sprouts = ", \
similarity[key_lime_bars, brussels_sprouts]
print "Similarity between key lime cookies and buffalo brussels sprouts = ", \
similarity[key_lime_cookies, brussels_sprouts]

Similarity between key lime bars and itself =  [ 1.]
Similarity between key lime cookies and itself =  [ 1.]
Similarity between buffalo brussels sprouts and itself =  [ 1.]
Similarity between key lime bars and key lime cookies =  [ 0.21586852]
Similarity between key lime bars and buffalo brussels sprouts =  [ 0.0846627]
Similarity between key lime cookies and buffalo brussels sprouts =  [ 0.08358824]


## Now find the five most similar recipes to given recipes

In [155]:
# import nearest neighbors module
from sklearn.neighbors import NearestNeighbors

num_interest = 3
# initialize the model
neigh = NearestNeighbors(n_neighbors = num_interest, algorithm='auto')
# fit the model
neigh.fit(tfidf_mat)
nearest_neighbors = neigh.kneighbors(return_distance=False)

In [156]:
from scipy import argwhere
query_names = ['Bacon Cheese Puff Balls', 'Mexican Flan', \
               'Spicy Buffalo Wings', 'Key Lime Bars', \
              'Key Lime Cookies', 'Cranberry Margarita']
closest_recipes = []
for recipe in query_names:
    query_ind = RD[RD['Name'] == recipe].index
    curr_recipe_close_recipes = []
    for i in xrange(num_interest):
        curr_ind = nearest_neighbors[query_ind,i]
        curr_recipe_close_recipes.append(Recipes_Names[curr_ind][0])
    
    closest_recipes.append(curr_recipe_close_recipes)




In [19]:
# display this data in a table
import matplotlib.pyplot as plt
%matplotlib inline

In [93]:
fig, ax = plt.subplots(1,1)
ax.axis('off')
colLabels =xrange(1,num_interest+1)
table = ax.table(cellText=closest_recipes, rowLabels=query_names,\
            colLabels=colLabels, loc='center')
table.auto_set_font_size(False)
table.set_fontsize(200)
table.scale(50, 50)

# Classification

The website "Gourmetsleuth.com" has categorized these recipes into 9 different courses. The dataframe we created stores these categories in the column 'Classification'. Let's see if we can implement a learning algorithm to classify these recipes.

In [97]:
RD.tail()

,Classification,Ingredients and Instructions,Name
967,8,potatoes of bacon chopped onion diced allpu...,Winter Creme Fraiche Baked Potato Soup
968,8,granulated sugar masa harina water milk ca...,Atole de Fresa (Strawberry Dessert Soup)
969,8,carrots peeled and cut in pieces leek white p...,Carrot Leek Soup
970,8,peppers stemmed and seeded butter finely dic...,Cream of Jalapeno Soup
971,8,unsweetend coconut milk palm sugar lemon gra...,Hawaiian Prawns In Coconut Curry Soup


In [107]:
# Look at the first few entries
RD.head()

,Classification,Ingredients and Instructions,Name
0,0,water butter salt pepper all purpose flour...,Bacon Cheese Puff Balls
1,0,dates pitted bacon lean cut in half widthwise...,Bacon Wrapped Sherried Dates
2,0,zucchini kosher salt french lentils very fr...,Baked Stuffed Portabellini Mushrooms
3,0,sour cream lower fat mayonnaise garlic mince...,Buffalo Brussels Sprouts
4,0,cans chicken broth boneless pork shoulder ch...,Carnitas


In [175]:
# The array below lists the categories
categories = ['Appetizers and Snacks', 'Bread, Muffins and Rolls', 'Breakfast and Brunch', 
                'Desserts', 'Drinks and Beverages', 'Main Dishes', 'Salads and Dressings',
             'Sides', 'Soups, Stews and Chili']

## Try to fit a logistic classifier

In [178]:
X = tfidf_mat
Y = RD['Classification'].as_matrix()

# first shuffle the entries
ind = np.random.permutation(num_recipes)

# separate into training and test set
train_num = int(round(.9*num_recipes))
train_ind = ind[:train_num]
test_ind = ind[train_num:]

train_X = X[train_ind,:]
train_Y = Y[train_ind]

test_X = X[test_ind,:]
test_Y = Y[test_ind]

print "num_train_data = ", len(train_Y)
print "num_test_data = ", len(test_Y)

num_train_data =  874
num_test_data =  97


In [168]:
from sklearn import linear_model
# create a logistic regression classifier that uses a cross-validation set to
# find the optimum C parameter
logreg = linear_model.LogisticRegressionCV(solver='liblinear', penalty='l1')

In [171]:
# fit the training data
logreg.fit(train_X, train_Y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0)

In [172]:
# Predict the classification on the test data
expected = test_Y
predicted = logreg.predict(test_X)

In [177]:
from sklearn import metrics
print("Classification report for classifier %s:\n%s\n"
     % (logreg, metrics.classification_report(expected, predicted)))
print ("The classification categories are as follows:")
num_categories = 9
for i in xrange(num_categories):
    print ("%d %s" % (i, categories[i]))
print("\nConfusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Classification report for classifier LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0):
             precision    recall  f1-score   support

        0.0       0.62      0.47      0.53        17
        1.0       1.00      1.00      1.00         2
        2.0       0.40      0.50      0.44         4
        3.0       0.75      1.00      0.86        15
        4.0       0.85      0.85      0.85        13
        5.0       0.80      0.50      0.62         8
        6.0       0.73      1.00      0.84         8
        7.0       0.67      0.64      0.65        22
        8.0       0.57      0.50      0.53         8

avg / total       0.70      0.70      0.69        97


The classification categories are as follows:
0 Appetizers and Snacks
1 Bre